In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
def clean_name(name):
    name = name.replace('(c)', '')
    name = name.replace('†', '')
    name = name.strip()
    return name

def find_wicket_c(wicket):
    wicket = wicket.replace('†', '')
    wicket = wicket.strip()
    if wicket.startswith('c & b '):
        wicket_c = wicket[len('c & b '):]
    elif wicket.startswith('c '):
        wicket_c = wicket[len('c '):wicket.find(' b ')]
    else:
        wicket_c = 'NA'
    return wicket_c

def find_wicket_b(wicket):
    wicket = wicket.replace('†', '')
    wicket = wicket.strip()
    if wicket.startswith('c & b '):
        wicket_b = wicket[len('c & b '):]
    elif wicket.startswith('b '):
        wicket_b = wicket[len('b '):]
    elif wicket.startswith('lbw b '):
        wicket_b = wicket[len('lbw b '):]
    elif wicket.startswith('c '):
        wicket_b = wicket[wicket.find(' b ')+3:]
    elif wicket.startswith('st '):
        wicket_b = wicket[wicket.find(' b ')+3:]
    else:
        wicket_b = 'NA'
    return wicket_b

def find_wicket_st(wicket):
    wicket = wicket.replace('†', '')
    wicket = wicket.strip()
    if wicket.startswith('st '):
        wicket_st = wicket[len('st '):wicket.find(' b ')]
    else:
        wicket_st = 'NA'
    return wicket_st

def find_wicket_ro(wicket):
    wicket = wicket.replace('†', '')
    wicket = wicket.strip()
    if wicket.startswith('run out ('):
        if "/" in wicket:
            wicket_ro_t, wicket_ro_c = wicket[len('run out ('):-1].split('/')[:2]
        else:
            wicket_ro_t, wicket_ro_c = wicket[len('run out ('):-1], wicket[len('run out ('):-1]
    else:
        wicket_ro_t, wicket_ro_c = 'NA', 'NA'
    return wicket_ro_t, wicket_ro_c

def match_player_caught(row, players_1, players_2):
    if row['wicket_c'] == 'NA':
        return 'NA'
    if row['inning'] == 1:
        players = players_2
    else:
        players = players_1
    for player in players:
        if row['wicket_c'] in player:
            return player
    else:
        return 'Not Matched'
        
def match_player_bowled(row, players_1, players_2):
    if row['wicket_b'] == 'NA':
        return 'NA'
    if row['inning'] == 1:
        players = players_2
    else:
        players = players_1
    for player in players:
        if row['wicket_b'] in player:
            return player
    else:
        return 'Not Matched'
        
def match_player_stumped(row, players_1, players_2):
    if row['wicket_st'] == 'NA':
        return 'NA'
    if row['inning'] == 1:
        players = players_2
    else:
        players = players_1
    for player in players:
        if row['wicket_st'] in player:
            return player
    else:
        return 'Not Matched'
        
def match_player_ro_throwed(row, players_1, players_2):
    if row['wicket_ro_t'] == 'NA':
        return 'NA'
    if row['inning'] == 1:
        players = players_2
    else:
        players = players_1
    for player in players:
        if row['wicket_ro_t'] in player:
            return player
    else:
        return 'Not Matched'
        
def match_player_ro_caught(row, players_1, players_2):
    if row['wicket_ro_c'] == 'NA':
        return 'NA'
    if row['inning'] == 1:
        players = players_2
    else:
        players = players_1
    for player in players:
        if row['wicket_ro_c'] in player:
            return player
    else:
        return 'Not Matched'

In [3]:
def create_player_df(match_dir):
    
    batsman_df = pd.read_csv(os.path.join(match_dir, 'batsman_df.csv'))
    if len(batsman_df) == 0:
        return pd.DataFrame(columns=['NA'])
    bowler_df = pd.read_csv(os.path.join(match_dir, 'bowler_df.csv'))
    bowler_df = bowler_df.rename(columns={
        'runs': 'runs_given',
        'fours': 'fours_given',
        'sixes': 'sixes_given'
    })
    if 'inning' in bowler_df.columns:
        bowler_df = bowler_df.drop(columns=['inning'])
    with open(os.path.join(match_dir, 'meta_data.json'), 'r') as file:
        meta_data = json.load(file)
    team_map = {1:meta_data['team_1'], 2:meta_data['team_2']}
        
    player_df = batsman_df[batsman_df['name'] != 'Extras'].copy()
    player_df['is_captain'] = player_df['name'].str.contains('(c)', regex=False, case=True)
    player_df['is_wicket_keeper'] = player_df['name'].str.contains('†', regex=False, case=True)
    player_df['name'] = player_df['name'].apply(clean_name)
    player_df['team'] = player_df['inning'].map(team_map)
    player_df['wicket_c'] = player_df['wicket'].apply(find_wicket_c)
    #player_df['wicket_b'] = player_df['wicket'].apply(find_wicket_b)
    player_df['wicket_st'] = player_df['wicket'].apply(find_wicket_st)
    player_df['wicket_ro_t'], player_df['wicket_ro_c']  = zip(*player_df['wicket'].apply(find_wicket_ro))
    
    players_1 = player_df[player_df['inning']==1]['name']
    players_2 = player_df[player_df['inning']==2]['name']
    player_df['player_caught'] = player_df.apply(match_player_caught, axis=1, 
                                                 players_1=players_1, players_2=players_2)
    #player_df['player_bowled'] = player_df.apply(match_player_bowled, axis=1, 
    #                                             players_1=players_1, players_2=players_2)
    player_df['player_stumped'] = player_df.apply(match_player_stumped, axis=1, 
                                                 players_1=players_1, players_2=players_2)
    player_df['player_ro_throwed'] = player_df.apply(match_player_ro_throwed, axis=1, 
                                                 players_1=players_1, players_2=players_2)
    player_df['player_ro_caught'] = player_df.apply(match_player_ro_caught, axis=1, 
                                                 players_1=players_1, players_2=players_2)
    
    player_df['caught_count'] = player_df['name'].map(player_df['player_caught'].value_counts())
    player_df['caught_count'] = player_df['caught_count'].fillna(0).astype('int')
    #player_df['bowled_count'] = player_df['name'].map(player_df['player_bowled'].value_counts())
    #player_df['bowled_count'] = player_df['bowled_count'].fillna(0).astype('int')
    player_df['stumped_count'] = player_df['name'].map(player_df['player_stumped'].value_counts())
    player_df['stumped_count'] = player_df['stumped_count'].fillna(0).astype('int')
    player_df['ro_throwed_count'] = player_df['name'].map(player_df['player_ro_throwed'].value_counts())
    player_df['ro_throwed_count'] = player_df['ro_throwed_count'].fillna(0).astype('int')
    player_df['ro_caught_count'] = player_df['name'].map(player_df['player_ro_caught'].value_counts())
    player_df['ro_caught_count'] = player_df['ro_caught_count'].fillna(0).astype('int')
        
    player_df = pd.merge(player_df, bowler_df, on='name', how='outer').fillna('NA')
    
    return player_df

In [4]:
def create_dream11_df(match_dir):
    
    player_df = pd.read_csv(os.path.join(match_dir, 'player_df.csv'))
    if len(player_df) == 0:
        return pd.DataFrame(columns=['NA'])
    player_df['strike_rate'] = player_df['strike_rate'].replace('-', -1).astype('float')
    player_df['runs'] = player_df['runs'].replace('-', 0).astype('float')
    player_df['wicket'] = player_df['wicket'].replace('absent hurt', 'Did not bat')
    player_df['economy'] = pd.to_numeric(player_df['economy'].replace('-', np.nan))
    dream11_df = player_df[['name', 'team']].copy()
    
    dream11_df['starting_11_actual'] = 'Yes'
    dream11_df['starting_11_points'] = 4
    
    dream11_df['runs_actual'] = player_df['runs']
    dream11_df['runs_points'] = dream11_df['runs_actual']
    
    dream11_df['fours_actual'] = player_df['fours']
    dream11_df['fours_points'] = dream11_df['fours_actual']
    
    dream11_df['sixes_actual'] = player_df['sixes']
    dream11_df['sixes_points'] = dream11_df['sixes_actual'] * 2
    
    atleast_10_balls = player_df['balls'] >= 10
    strike_rate_points = pd.to_numeric(pd.cut(player_df['strike_rate'],
                                bins=[0, 50, 60, 70], 
                                right=False, 
                                labels=[-6, -4, -2])).fillna(0)
    dream11_df['atleast_10_balls'] = atleast_10_balls
    dream11_df['strike_rate_actual'] = player_df['strike_rate']
    dream11_df['strike_rate_points'] = dream11_df['atleast_10_balls'] * strike_rate_points 
    
    points_50 = (player_df['runs'] >= 50) * 8
    points_100 = (player_df['runs'] >= 100) * 8
    dream11_df['50_100_points'] = points_50 + points_100
    
    dream11_df['duck_actual'] = (player_df['runs'] == 0) & (player_df['wicket'] != 'Did not bat')
    dream11_df['duck_points'] = dream11_df['duck_actual'] * -2
    
    dream11_df['wickets_actual'] = player_df['wickets'].replace('NA', 0).fillna(0)
    dream11_df['wickets_points'] = dream11_df['wickets_actual'] * 25
    
    dream11_df['maidens_actual'] = player_df['maidens'].replace('NA', 0).fillna(0)
    dream11_df['maidens_points'] = dream11_df['maidens_actual'] * 8
    
    atleast_2_overs = player_df['overs'] >= 2
    economy_rate_points = pd.to_numeric(pd.cut(player_df['economy'],
                                bins=[0, 4, 5, 6.01, 9, 10.01, 11.01, 36], 
                                right=False, 
                                labels=[6, 4, 2, 0, -2, -4, -6])).fillna(0)
    dream11_df['atleast_2_overs'] = atleast_2_overs
    dream11_df['economy_rate_actual'] = player_df['economy']
    dream11_df['economy_rate_points'] = dream11_df['atleast_2_overs'] * economy_rate_points 
    
    wickets_4 = (player_df['wickets'] >= 4) * 8
    wickets_4 = (player_df['wickets'] >= 5) * 8
    dream11_df['bonus_points'] = wickets_4 + wickets_4
    
    dream11_df['catches_actual'] = player_df['caught_count'] 
    dream11_df['catches_points'] = dream11_df['catches_actual'] * 8
    
    dream11_df['run_out_actual'] = player_df['ro_throwed_count'] + player_df['ro_caught_count']
    dream11_df['run_out_points'] = dream11_df['run_out_actual'] * 6
    
    dream11_df['stumping_actual'] = player_df['stumped_count']
    dream11_df['stumping_points'] = dream11_df['stumping_actual'] * 12
    
    points_cols = [col for col in dream11_df.columns if col.endswith('_points')]
    dream11_df['total_points'] = dream11_df[points_cols].sum(axis=1)
    
    dream11_df = dream11_df.sort_values(by='total_points', ascending=False).reset_index(drop=True)
    dream11_df['rank'] = dream11_df.index + 1
    
    cols = list(dream11_df.columns)
    cols.remove('total_points')
    cols = ['total_points'] + cols
    dream11_df = dream11_df[cols]
    
    return dream11_df

In [5]:
data_dir = 'Data'
seasons = range(2010, 2021)

for season in tqdm(seasons, desc='seasons'):
    season_dir = os.path.join(data_dir, str(season))
    matches = sorted(os.listdir(season_dir))
    for match in tqdm(matches, desc='matches', leave=False):
        match_dir = os.path.join(season_dir, match)
        try:
            player_df = create_player_df(match_dir)
        except Exception as e:      
            print(0, match_dir)
            print(e)
            raise
        player_df.to_csv(os.path.join(match_dir, 'player_df.csv'), index=False)
        
        try:
            dream11_df = create_dream11_df(match_dir)
        except Exception as e:         
            print(1, match_dir)
            print(e)
            raise
        dream11_df.to_csv(os.path.join(match_dir, 'dream11_df.csv'), index=False)

### Merging all dream11 tables

In [8]:
data_dir = 'Data'
seasons = range(2010, 2021)

dream11_df = pd.DataFrame()
for season in tqdm(seasons, desc='seasons'):
    season_dir = os.path.join(data_dir, str(season))
    matches = sorted(os.listdir(season_dir))
    for match in tqdm(matches, desc='matches', leave=False):
        match_dir = os.path.join(season_dir, match)
        with open(os.path.join(match_dir, 'meta_data.json'), 'r') as file:
            meta_data = json.load(file)
        match_dream11_df = pd.read_csv(os.path.join(match_dir, 'dream11_df.csv'))
        cols = list(match_dream11_df.columns)
        if len(match_dream11_df):
            match_dream11_df['season_id'] = season
            match_dream11_df['match_id'] = meta_data['match_id']
            cols = ['season_id', 'match_id'] + cols
            match_dream11_df = match_dream11_df[cols]
            dream11_df = pd.concat([dream11_df, match_dream11_df], ignore_index=True)
dream11_df.to_csv(os.path.join(data_dir, 'dream11_df.csv'), index=False)